In [5]:
import tensorflow as tf
tf.__version__

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# Creates a session with device placement logs
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
print(sess)

In [ ]:
# GPU for training.
DEVICE = "/gpu:0" 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import random
import math
import re
import time
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils

# Root directory of the project
#ROOT_DIR = "D:\MRCNN_tensorflow2.7_env\Mask-RCNN"
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library


# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")


In [ ]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"


    # NUMBER OF GPUs to use. When using only a CPU, this needs to be set to 1.
    GPU_COUNT = 1
    
    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 4
    
    # Number of classes (including background)
    NUM_CLASSES = 1 + 11 # Background + Hard_hat, Safety_vest

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 5

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

In [ ]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):
        """Load a subset of the Dog-Cat dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
      
        self.add_class("object", 1, "Bridal Long Shirt")
        self.add_class("object", 2, "Bridal Dupatta")
        self.add_class("object", 3, "Bridal Choli")
        self.add_class("object", 4, "Bridal Lehenga")
        self.add_class("object", 5,"Wedding Guest Lehenga")
        self.add_class("object", 6,"Wedding Guest Dupatta")
        self.add_class("object", 7,"Wedding Guest Choli")
        self.add_class("object", 8,"Bridal Maxi")
        self.add_class("object", 9,"Wedding Guest Long Shirt")
        self.add_class("object", 10,"Bridal Peplum")
        self.add_class("object", 11,"Wedding Guest Maxi")
        
        
        # Train or validation dataset?
        assert subset in ["train", "val","test"]
        dataset_dir = os.path.join(dataset_dir, subset)

        if subset == "train":
            annotations1 = json.load(open(r'/content/Threads-of-Fashion/dataset/train/train.json'))
        elif subset == "val":
            annotations1 = json.load(open(r'/content/Threads-of-Fashion/dataset/val/val.json'))
        elif subset == "test":
            annotations1 = json.load(open(r'/content/Threads-of-Fashion/dataset/test/test.json'))
            

        annotations = list(annotations1.values())
        print(annotations1)
       
          # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:

            polygons = [r['shape_attributes'] for r in a['regions'].values()] 
            objects = [s['region_attributes'] for s in a['regions'].values()]
            print("objects:",objects)
            num_ids=[]
            

            for n in objects:

                try:

                    if n['name']== "Bridal Long Shirt":
                        num_ids.append(1)

                    elif n['name']=="Bridal Dupatta":
                         num_ids.append(2)
                    
                    elif n['name']=="Bridal Choli":
                         num_ids.append(3)

                    elif n['name']=="Bridal Lehenga":
                         num_ids.append(4)
                    
                    elif n['name']=="Wedding Guest Lehenga":
                         num_ids.append(5)
                    
                    elif n['name']=="Wedding Guest Dupatta":
                         num_ids.append(6)

                    elif n['name']=="Wedding Guest Choli":
                         num_ids.append(7)
                    
                    elif n['name']=="Bridal Maxi":
                         num_ids.append(8)
                    elif n['name']=="Wedding Guest Long Shirt":
                         num_ids.append(9)
                    
                    elif n['name']=="Bridal Peplum":
                         num_ids.append(10)
                    
                    elif n['name']=="Wedding Guest Maxi":
                         num_ids.append(11)
                    
                except:
                    
                    pass
                
            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Dog-Cat dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
class InferenceConfig(CustomConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    #Minimum probability value to accept a detected instance
    # ROIs below this threshold are skipped
    DETECTION_MIN_CONFIDENCE = 0.7

    # Non-maximum suppression threshold for detection
    DETECTION_NMS_THRESHOLD = 0.5

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=DEFAULT_LOGS_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")


#model_path = model.find_last()
model_path =r"/content/Threads-of-Fashion/logs/object20240128T1624/mask_rcnn_object_0005.h5"


# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)




In [ ]:
# Training dataset.
dataset_train = CustomDataset()
dataset_train.load_custom(r"/content/Threads-of-Fashion/dataset", "train")
dataset_train.prepare()

# Validation dataset
dataset_val = CustomDataset()
dataset_val.load_custom(r"/content/Threads-of-Fashion/dataset", "val")
dataset_val.prepare()

# Testing on Validation Dataset

In [ ]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

# Calculating Mean Average Precision on Validation Dataset

In [ ]:
config=inference_config
dataset = dataset_val


gt_tot = np.array([])
pred_tot = np.array([])

#mAP list
mAP_ = []

#compute gt_tot, pred_tot and mAP for each image in the test dataset
for image_id in dataset.image_ids:
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config, image_id)#, #use_mini_mask=False)
    info = dataset.image_info[image_id]

    # Run the model
    results = model.detect([image], verbose=1)
    r = results[0]
    
    #compute gt_tot and pred_tot
    gt, pred = utils.gt_pred_lists(gt_class_id, gt_bbox, r['class_ids'], r['rois'])
    gt_tot = np.append(gt_tot, gt)
    pred_tot = np.append(pred_tot, pred)
    
    #precision_, recall_, AP_ 
    AP_, precision_, recall_, overlap_ = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])
    #check if the vectors len are equal
    print("the actual len of the gt vect is : ", len(gt_tot))
    print("the actual len of the pred vect is : ", len(pred_tot))
    
    mAP_.append(AP_)
    print("Average precision of this image : ",AP_)
    print("The actual mean average precision for the whole images", sum(mAP_)/len(mAP_))
    #print("Ground truth object : "+dataset.class_names[gt])

    #print("Predicted object : "+dataset.class_names[pred])
    # for j in range(len(dataset.class_names[gt])):
	    # print("Ground truth object : "+j)
import pandas as pd
gt_tot=gt_tot.astype(int)
pred_tot=pred_tot.astype(int)
#save the vectors of gt and pred
save_dir = "output"
gt_pred_tot_json = {"gt_tot" : gt_tot, "pred_tot" : pred_tot}
df = pd.DataFrame(gt_pred_tot_json)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
df.to_json(os.path.join(save_dir,"gt_pred_val.json"))


# Plotting Confusion Matrix of Inference on Validation Dataset

In [ ]:
# tp,fp,fn=utils.plot_confusion_matrix_from_data(gt_tot,pred_tot,
#                                                columns=["Bridal Long Shirt",
#                                                         "Bridal Dupatta",
#                                                         "Bridal Choli",
#                                                         "Bridal Lehenga",
#                                                         "Wedding Guest Lehenga",
#                                                         "Wedding Guest Dupatta",
#                                                         "Wedding Guest Choli",
#                                                         "Bridal Maxi",
#                                                         "Wedding Guest Long Shirt",
#                                                         "Bridal Peplum",
#                                                         "Wedding Guest Maxi"] ,
#                                                fz=18, figsize=(20,20), lw=0.5)



import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

cm = confusion_matrix(gt_tot, pred_tot)

class_names = ["Bridal Long Shirt", "Bridal Dupatta", "Bridal Choli", "Bridal Lehenga", 
               "Wedding Guest Lehenga", "Wedding Guest Dupatta", "Wedding Guest Choli", 
               "Bridal Maxi", "Wedding Guest Long Shirt", "Bridal Peplum", "Wedding Guest Maxi"]

def plot_confusion_matrix(cm, classes=class_names,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.figure(figsize=(11, 11))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


print(cm.shape)
plot_confusion_matrix(cm, classes=class_names)
plt.show()


In [ ]:
# tp = []
# fp = []
# fn = []

# # Loop through each class (excluding background class)
# for i in range(len(class_names)):
#     # True positives (TP): Diagonal elements of the confusion matrix
#     tp.append(cm[i, i])
    
#     # False positives (FP): Sum of values in the corresponding column, excluding TP
#     fp.append(np.sum(cm[:, i]) - cm[i, i])
    
#     # False negatives (FN): Sum of values in the corresponding row, excluding TP
#     fn.append(np.sum(cm[i, :]) - cm[i, i])

# # Print TP, FP, FN for each class
# print("TP for each class:", tp)
# print("FP for each class:", fp)
# print("FN for each class:", fn)

# Precision Recall Curve

In [ ]:
# Draw precision-recall curve

AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])
visualize.plot_precision_recall(AP, precisions, recalls)

# Testing on Test Dataset

In [ ]:
import skimage
real_test_dir = r'/content/Threads-of-Fashion/dataset/test'
image_paths = []
for filename in os.listdir(real_test_dir):
    if os.path.splitext(filename)[1].lower() in ['.png', '.jpg', '.jpeg']:
        image_paths.append(os.path.join(real_test_dir, filename))

for image_path in image_paths:
    img = skimage.io.imread(image_path)
    img_arr = np.array(img)
    results = model.detect([img_arr], verbose=1)
    r = results[0]
    visualize.display_instances(img, r['rois'], r['masks'], r['class_ids'], 
                                dataset_val.class_names, r['scores'], figsize=(5,5))

In [ ]:
# testing dataset
dataset_test = CustomDataset()
dataset_test.load_custom(r"/content/Threads-of-Fashion/dataset", "test")
dataset_test.prepare()

# Calculating Mean Average Precision on Test Dataset

In [ ]:
config=inference_config
dataset = dataset_test


gt_tot = np.array([])
pred_tot = np.array([])

#mAP list
mAP_ = []

#compute gt_tot, pred_tot and mAP for each image in the test dataset
for image_id in dataset.image_ids:
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset, config, image_id)#, #use_mini_mask=False)
    info = dataset.image_info[image_id]

    # Run the model
    results = model.detect([image], verbose=1)
    r = results[0]
    
    #compute gt_tot and pred_tot
    gt, pred = utils.gt_pred_lists(gt_class_id, gt_bbox, r['class_ids'], r['rois'])
    gt_tot = np.append(gt_tot, gt)
    pred_tot = np.append(pred_tot, pred)
    
    #precision_, recall_, AP_ 
    AP_, precision_, recall_, overlap_ = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])
    #check if the vectors len are equal
    print("the actual len of the gt vect is : ", len(gt_tot))
    print("the actual len of the pred vect is : ", len(pred_tot))
    
    mAP_.append(AP_)
    print("Average precision of this image : ",AP_)
    print("The actual mean average precision for the whole images", sum(mAP_)/len(mAP_))
    #print("Ground truth object : "+dataset.class_names[gt])

    #print("Predicted object : "+dataset.class_names[pred])
    # for j in range(len(dataset.class_names[gt])):
	    # print("Ground truth object : "+j)
import pandas as pd
gt_tot=gt_tot.astype(int)
pred_tot=pred_tot.astype(int)
#save the vectors of gt and pred
save_dir = "output"
gt_pred_tot_json = {"gt_tot" : gt_tot, "pred_tot" : pred_tot}
df = pd.DataFrame(gt_pred_tot_json)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
df.to_json(os.path.join(save_dir,"gt_pred_test.json"))


# Plotting Confusion Matrix of Inference on Test Dataset

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

cm = confusion_matrix(gt_tot, pred_tot)

class_names = ["Bridal Long Shirt", "Bridal Dupatta", "Bridal Choli", "Bridal Lehenga", 
               "Wedding Guest Lehenga", "Wedding Guest Dupatta", "Wedding Guest Choli", 
               "Bridal Maxi", "Wedding Guest Long Shirt", "Bridal Peplum", "Wedding Guest Maxi"]

def plot_confusion_matrix(cm, classes=class_names,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.figure(figsize=(11, 11))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')



plot_confusion_matrix(cm, classes=class_names)
plt.show()


In [ ]:
# tp = []
# fp = []
# fn = []

# # Loop through each class (excluding background class)
# for i in range(len(class_names)):
#     # True positives (TP): Diagonal elements of the confusion matrix
#     tp.append(cm[i, i])
    
#     # False positives (FP): Sum of values in the corresponding column, excluding TP
#     fp.append(np.sum(cm[:, i]) - cm[i, i])
    
#     # False negatives (FN): Sum of values in the corresponding row, excluding TP
#     fn.append(np.sum(cm[i, :]) - cm[i, i])

# # Print TP, FP, FN for each class
# print("TP for each class:", tp)
# print("FP for each class:", fp)
# print("FN for each class:", fn)

# Precision Recall Curve

In [ ]:
# Draw precision-recall curve

AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                                          r['rois'], r['class_ids'], r['scores'], r['masks'])
visualize.plot_precision_recall(AP, precisions, recalls)